In [5]:
from math import floor
from statistics import mean
from scipy.optimize import brentq

In [2]:
race_length = 14400
long_stops = 2

In [ ]:
# GT3
base_pitstop_loss = 28
# TODO: Separate losses for in, outlap? Would be messy because of finish line location varying between tracks, along with fact that some pit boxes are before finish line, some after
fuel_tank_size = 100
refuel_rate = 0.353  # sec / L
tire_swap_time = 30

In [6]:
def estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint = None):
    global race_length
    global long_stops
    global base_pitstop_loss

    global fuel_tank_size
    global refuel_rate
    global tire_swap_time

    long_stop_time = long_stops * 180
    effective_race_length = race_length - long_stop_time

    if laps_per_stint:
        liters_to_refuel = laps_per_stint * average_fuel_consumption
    else:
        liters_to_refuel = fuel_tank_size - average_fuel_consumption  # To get a lower bound on max_stint_length, we lower bound pitstop length by
        # assuming there is at least one lap of fuel left in the tank when coming in

    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * liters_to_refuel)

    if not laps_per_stint:
        laps_per_stint = floor(fuel_tank_size / average_fuel_consumption)  # TODO Can this be moved up to avoid having the previous if laps_per_stint block?
    
    max_stint_length = average_lap_time * laps_per_stint + pitstop_length
    # TODO: Parameter for max laps out? (tire strategy, may be better to save for new strat calculator)

    stint_total = (effective_race_length + average_lap_time) / max_stint_length  # Adding average_lap_time to numerator accounts for 6 hours + 1 lap
    # We want an upper bound of this
    # pitstop_length is the sum of the time lost between the inlap and outlap. The first stint does not have an outlap but rather a formation lap (which is longer than an outlap),
    # the final stint has an outlap but not an inlap. Therefore right now stint_total is overestimated by outlap_differential / max_stint_length

    return stint_total, max_stint_length  # Separate function for max_stint_length?



In [20]:
def fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops):
    global fuel_tank_size
    stint_total = num_pitstops + 1
    obj = lambda laps_per_stint: estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)[0] - stint_total

    # Calculate optimal laps per stint
    optimal_laps_per_stint = brentq(obj, 1, floor(fuel_tank_size / average_fuel_consumption))

    # Return liters required to do optimal laps
    return average_fuel_consumption * optimal_laps_per_stint

In [8]:
estimate_stint_length(103.458, 2.9)

(3.9508540068583353, 3579.8483)

In [9]:
estimate_stint_length(102, 3.0)

(4.125147561096201, 3428.241)

In [ ]:
fuel_per_stint(102, 3.0, 4)  # L per refuel

81.48235294117649

In [4]:
car_1 = [(93, 2.55), (92, 2.7), (90.5, 2.65)]  # Rake, Dadeler
car_2 = [(92, 2.7), (93.4, 2.61), (91.7, 2.6)]  # Dadeler, Marcus, Marv
car_2_mod = car_2[1:]

print(f"Car 1 stints: {mean([estimate_stint_length(*x)[0] for x in car_1])}")
print(f"Car 2 stints: {mean([estimate_stint_length(*x)[0] for x in car_2])}")
print(f"Car 2 stints without dadeler: {mean([estimate_stint_length(*x)[0] for x in car_2_mod])}")

Car 1 stints: 6.008526530400015
Car 2 stints: 5.970472025708266
Car 2 stints without dadeler: 5.90643506426124
